In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score  #交叉验证评分
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error #均方根误差
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv('../data/train_chuli.csv')

In [50]:
# 输出数据预览
print(dataset.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  \
0            1         0       3    1  22.0      1      0   7.2500   
1            2         1       1    0  38.0      1      0  71.2833   
2            3         1       3    0  26.0      0      0   7.9250   
3            4         1       1    0  35.0      1      0  53.1000   
4            5         0       3    1  35.0      0      0   8.0500   

   family_size  Embarked_C  Embarked_Q  Embarked_S  
0            2           0           0           1  
1            2           1           0           0  
2            1           0           0           1  
3            2           0           0           1  
4            1           0           0           1  


In [51]:
from sklearn.model_selection import train_test_split #划分数据集
# 准备训练数据
x = dataset.drop(['Survived'], axis = 1)
y = dataset['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(623, 11) (268, 11) (623,) (268,)


In [52]:
#评价指标
import numpy as np

# 计算混淆矩阵
def compute_confusion_matrix(precited,expected):
    part = precited ^ expected             # 对结果进行分类，亦或使得判断正确的为0,判断错误的为1
    pcount = np.bincount(part)             # 分类结果统计，pcount[0]为0的个数，pcount[1]为1的个数
    tp_list = list(precited & expected)    # 将TP的计算结果转换为list
    fp_list = list(precited & ~expected)   # 将FP的计算结果转换为list
    tp = tp_list.count(1)                  # 统计TP的个数
    fp = fp_list.count(1)                  # 统计FP的个数
    tn = pcount[0] - tp                    # 统计TN的个数
    fn = pcount[1] - fp                    # 统计FN的个数
    return tp, fp, tn, fn

# 计算常用指标
def compute_indexes(tp, fp, tn, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)     # 准确率
    precision = tp / (tp+fp)               # 精确率
    recall = tp / (tp+fn)                  # 召回率
    F1 = (2*precision*recall) / (precision+recall)    # F1
    return accuracy, precision, recall, F1


In [53]:
from matplotlib.cm import rainbow 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
#决策树
rf_scores = []
dt = DecisionTreeClassifier(random_state=2)
rf_classifier = BaggingClassifier(base_estimator=dt, n_estimators=100, random_state=1)
rf_classifier.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=2),
                  n_estimators=100, random_state=1)

BaggingClassifier
- base_estimator : 对象或无，可选（默认=无）
基本估计量适合数据集的随机子集。如果为None，则基本估计量为决策树。
- n_estimators : int，可选（默认值为10）
集合中的基本估计量。
- max_samples : int或float，可选（默认值= 1.0）
从X抽取以训练每个基本估计量的样本数。
如果为int，则抽取样本 max_samples。
如果float，则抽取本 max_samples * X.shape[0]
max_features : int或float，可选（默认值= 1.0）
从X绘制以训练每个基本估计量的要素数量。
如果为int，则绘制特征 max_features。
如果是浮动的，则绘制特征 max_features * X.shape[1]
bootstrap : 布尔值，可选（默认= True）
是否抽取样本进行替换。如果为False，则执行不替换的采样。
oob_score : 布尔变量，可选（默认为False）
是否使用现成的样本来估计泛化误差。
warm_start : 布尔变量，可选（默认= False）
设置为True时，请重用上一个调用的解决方案以适合并为集合添加更多估计量，否则，仅适合一个全新的集合。
n_jobs : int或None（可选）（默认为None）
fit和 并行运行的作业数predict。None除非joblib.parallel_backend上下文中，否则表示1 。-1表示使用所有处理器。
random_state : 整型，RandomState实例或无，可选（默认值：无）
如果为int，则random_state是随机数生成器使用的种子；否则为false。

In [54]:
rfc_score = rf_classifier.score(x_test, y_test)#测试集准确率,j即accuracy
expected = y_test.astype('int32')#预期值
# print(type(expected))
# print(expected)
rfc_pre=rf_classifier.predict(x_test)
precited = rfc_pre.astype('int32')#模型预测值
# print(type(precited))
tp, fp, tn, fn = compute_confusion_matrix(precited, expected)
accuracy, precision, recall, F1 = compute_indexes(tp, fp, tn, fn)
rfc_pre = rf_classifier.predict(x_test)
rfc_auc = metrics.roc_auc_score(y_test, rfc_pre)
rfc_rmse = np.sqrt(mean_squared_error(y_test, rfc_pre))
print('TP FP TN FN')
print('%d %d %d %d'%(tp,fp,tn,fn))
print('accuracy precision recall F1')
print('%.4lf %.4lf %.4lf %.4lf'%(accuracy, precision, recall, F1))
print('auc rmse')
print('%.4lf %.4lf'%(rfc_auc, rfc_rmse))

TP FP TN FN
71 24 144 29
accuracy precision recall F1
0.8022 0.7474 0.7100 0.7282
auc rmse
0.7836 0.4447
